# Image Conversion Pipeline
- Taken from `https://github.com/thongonary/LCDJets/blob/master/Transform.py`
- For each row, produce an image from a matrix of features 

In [ ]:
features = sqlContext.read.format("parquet").load("file:/Users/vk/data/ML_MP_JR/ttbar_lepFilter_13TeV/*.parquet")

In [3]:
features.show()

+--------------------+
|            features|
+--------------------+
|[WrappedArray(44....|
|[WrappedArray(67....|
|[WrappedArray(189...|
|[WrappedArray(36....|
|[WrappedArray(31....|
|[WrappedArray(60....|
|[WrappedArray(386...|
|[WrappedArray(76....|
|[WrappedArray(44....|
|[WrappedArray(65....|
|[WrappedArray(51....|
|[WrappedArray(121...|
|[WrappedArray(43....|
|[WrappedArray(292...|
|[WrappedArray(705...|
|[WrappedArray(70....|
|[WrappedArray(319...|
|[WrappedArray(53....|
|[WrappedArray(48....|
|[WrappedArray(80....|
+--------------------+
only showing top 20 rows



## Step1 Build up the functionality
- various imports
- conversion function

In [5]:
import time
import numpy as np
from pyspark.sql import Row
import math

In [132]:
def convert2image(row):
    """Assume that a row contains a non-empty 2D matrix of features"""
    mat = np.asarray(row.features, dtype=np.float64)
    reduced = np.asarray(np.zeros((mat.shape[0], 4)))
    reduced[:, 0] = mat[:, 5]
    reduced[:, 1] = mat[:, 6]
    reduced[:, 2] = np.minimum(np.log(np.maximum(mat[:, 4], 1.001))/5., 10)
    reduced[:, 3] = np.argmax(mat[:, 13:], axis=-1)
    print reduced
    
    return Row(image=reduced.tolist())

In [133]:
before = time.time()
images = features\
    .rdd\
    .map(convert2image)\
    .toDF()
after = time.time()
print("Total Processing Time: %d seconds" % int(after - before))

Total Processing Time: 0 seconds


In [130]:
images.printSchema()

root
 |-- image: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



In [131]:
images.show()

+--------------------+
|               image|
+--------------------+
|[WrappedArray(0.6...|
|[WrappedArray(0.2...|
|[WrappedArray(1.6...|
|[WrappedArray(-0....|
|[WrappedArray(-0....|
|[WrappedArray(-1....|
|[WrappedArray(-1....|
|[WrappedArray(1.4...|
|[WrappedArray(-0....|
|[WrappedArray(1.3...|
|[WrappedArray(0.2...|
|[WrappedArray(1.8...|
|[WrappedArray(-0....|
|[WrappedArray(-2....|
|[WrappedArray(2.9...|
|[WrappedArray(-0....|
|[WrappedArray(0.6...|
|[WrappedArray(0.8...|
|[WrappedArray(0.2...|
|[WrappedArray(-0....|
+--------------------+
only showing top 20 rows

